In [ ]:
import os
import numpy as np
import math
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [38]:
def carregar_ficheiros_MEMM(pasta):
    sequencias = []
    for nome_ficheiro in os.listdir(pasta):
        if nome_ficheiro.endswith('.il'):  # <--- aqui está o problema corrigido
            caminho = os.path.join(pasta, nome_ficheiro)
            with open(caminho, 'r', encoding='utf-8') as f:
                tokens_estado = []
                for linha in f:
                    tokens = linha.strip().split()
                    for token in tokens[::-1]:  # inversão da ordem dos tokens
                        if "::" in token:
                            partes = token.split("::")
                            if len(partes) == 2:
                                obs, estado = partes
                                tokens_estado.append((obs, estado))
                if tokens_estado:
                    sequencias.append(tokens_estado)
    return sequencias

In [39]:
def extrair_features_MEMM(sequencias):
    X, y = [], []
    for seq in sequencias:
        estado_anterior = "START"
        for observacao, estado in seq:
            features = {
                'observacao': observacao,
                'estado_anterior': estado_anterior
            }
            X.append(features)
            y.append(estado)
            estado_anterior = estado
    return X, y

In [40]:
def treinar_MEMM(X, y):
    modelo = Pipeline([
        ('vectorizer', DictVectorizer(sparse=False)),
        ('classifier', LogisticRegression(max_iter=300, class_weight='balanced'))
    ])
    modelo.fit(X, y)
    return modelo

In [41]:
def probs_MEMM(modelo, observacao, estado_anterior, etiquetas):
    features = {
        'observacao': observacao,
        'estado_anterior': estado_anterior
    }
    proba = modelo.predict_proba([features])[0]
    etiqueta_proba = dict(zip(modelo.classes_, proba))
    return [etiqueta_proba.get(et, 1e-6) for et in etiquetas]  # evitar log(0)

In [ ]:
def viterbi_MEMM(modelo, observacoes, etiquetas):
    T = len(observacoes)
    N = len(etiquetas)
    dp = np.zeros((T, N))
    backpointer = np.zeros((T, N), dtype=int)

    for j, s in enumerate(etiquetas):
        dp[0][j] = np.log(probs_MEMM(modelo, observacoes[0], 'START', etiquetas)[j])

    for t in range(1, T):
        for j, s in enumerate(etiquetas):
            max_prob = float('-inf')
            max_state = 0
            for i, s_prev in enumerate(etiquetas):
                prob = dp[t - 1][i] + np.log(probs_MEMM(modelo, observacoes[t], s_prev, etiquetas)[j])
                if prob > max_prob:
                    max_prob = prob
                    max_state = i
            dp[t][j] = max_prob
            backpointer[t][j] = max_state

    best_path = []
    last_state = np.argmax(dp[-1])
    best_path.append(etiquetas[last_state])
    for t in range(T - 1, 0, -1):
        last_state = backpointer[t][last_state]
        best_path.insert(0, etiquetas[last_state])

    return best_path

In [ ]:
# Carrega os dados
sequencias = carregar_ficheiros_MEMM("PHP2IL/hmm/vuln_grupo_alto_last_ANN")
X, y = extrair_features_MEMM(sequencias)

# Treina modelo
modelo_MEMM = treinar_MEMM(X, y)
etiquetas_possiveis = sorted(list(set(y)))

# Previsão com Viterbi para uma nova sequência
nova_seq_obs = ['var', 'GET', 'mysql_real_escape_string', 'var']
estados_previstos = viterbi_MEMM(modelo_MEMM, nova_seq_obs, etiquetas_possiveis)

print("Observações:", nova_seq_obs)
print("Estados preditos:", estados_previstos)

ValueError: Sample sequence X is empty.

In [ ]:
sequencias = carregar_ficheiros_MEMM("PHP2IL/hmm/vuln_grupo_alto_last_ANN")
print(f"Total de sequências carregadas: {len(sequencias)}")

Total de sequências carregadas: 0
